Certainly! Below is a comprehensive PyTorch tutorial that demonstrates the use of several key features: `autograd`, `DataLoader`, `TensorBoard`, `profiler`, `TorchScript`, `quantization`, and `Captum`. We'll build a simple neural network to classify the MNIST dataset.

### Step 1: Setup and Imports

First, let's import the necessary libraries and set up the environment.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import torch.autograd.profiler as profiler
import torch.quantization
from captum.attr import IntegratedGradients
import torch.jit

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu




### Step 2: Define the Model

We'll define a simple feedforward neural network for MNIST classification.

In [3]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN().to(device)




### Step 3: Data Loading

We'll use the `DataLoader` to load the MNIST dataset.


In [4]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1006)>



100%|██████████| 9912422/9912422 [00:06<00:00, 1587678.38it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1006)>



100%|██████████| 28881/28881 [00:00<00:00, 271254.38it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1006)>



100%|██████████| 1648877/1648877 [00:03<00:00, 532276.25it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1006)>



100%|██████████| 4542/4542 [00:00<00:00, 1177557.72it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw





### Step 4: Training Loop

We'll define a simple training loop and use `autograd` for backpropagation.



In [5]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

for epoch in range(2):  # Train for 2 epochs
    loss = train(model, train_loader, criterion, optimizer, device)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}')


Epoch 1, Loss: 0.2561
Epoch 2, Loss: 0.1129




### Step 5: TensorBoard Integration

We'll use TensorBoard to visualize the training process.



In [7]:

writer = SummaryWriter('runs/mnist_experiment')

# Log the model graph
dataiter = iter(train_loader)
images, labels = next(dataiter)
writer.add_graph(model, images.to(device))

# Log training loss
for epoch in range(2):
    loss = train(model, train_loader, criterion, optimizer, device)
    writer.add_scalar('training loss', loss, epoch)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}')

writer.close()


RuntimeError: required keyword attribute 'name' has the wrong type

In [ ]:


### Step 6: Profiling

We'll use the profiler to analyze the performance of our model.

```python
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
```

### Step 7: TorchScript

We'll convert our model to TorchScript for deployment.

```python
scripted_model = torch.jit.script(model)
scripted_model.save("mnist_model.pt")
```

### Step 8: Quantization

We'll quantize our model to reduce its size and improve inference speed.

```python
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)

# Save the quantized model
torch.jit.save(torch.jit.script(quantized_model), "mnist_quantized_model.pt")
```

### Step 9: Captum for Model Interpretability

We'll use Captum to interpret the model's predictions.

```python
model.eval()
ig = IntegratedGradients(model)

# Get a sample image
dataiter = iter(test_loader)
images, labels = dataiter.next()
image = images[0].unsqueeze(0).to(device)

# Compute attributions
attributions, delta = ig.attribute(image, target=labels[0], return_convergence_delta=True)

print('Attributions:', attributions)
print('Convergence Delta:', delta)
```

### Conclusion

This tutorial demonstrated how to use several key features of PyTorch, including `autograd`, `DataLoader`, `TensorBoard`, `profiler`, `TorchScript`, `quantization`, and `Captum`. Each of these features plays a crucial role in developing, optimizing, and interpreting deep learning models.